In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
stop = stopwords.words('russian')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df =  pd.read_csv(r"C:\Users\coolz\assignments\data.csv", names = ['Review', 'Rating'], sep=',')
df = df.dropna(how='any',axis=0)
np.random.seed(37)
df = df.drop_duplicates(subset=['Review', 'Rating'], keep=False)
df = df.reset_index(drop=True)

In [3]:
df['Rating']=df['Rating'].astype(int)
df=df[df['Rating']!=3]
df['label']=np.where(df['Rating']>=3,1,0)

In [4]:
df['Rating'].value_counts()

5    107885
4     50785
2     16121
1     15511
Name: Rating, dtype: int64

In [24]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle
data=df[df['label']==0][:30000]
data=data.append(df[df['label']==1][:150000])
data = data.reset_index(drop=True)
display(data['label'].value_counts())
data

1    150000
0     30000
Name: label, dtype: int64

,Review,Rating,label
0,"купила телефон у офицалов 2 апреля 2011 года, ...",2,0
1,может мне попалась именно бракованная модельне...,1,0
2,"Покупал для игр , интернета , видео . Игры - ,...",2,0
3,НЕ БЕРИТЕ!!! до тех пор пока не сделают съёмну...,1,0
4,цена-качесво 100%. если решат проблему с тупен...,2,0
...,...,...,...
179995,"Относительно ""небольшой"" для ребенка размер, т...",5,1
179996,"Покупался в подарок, перед вручением попользов...",5,1
179997,Пользуюсь смартфоном 3 недели. Ждал этого смар...,5,1
179998,плохо что сильно завязан на сотовый интернет. ...,4,1


In [25]:
data['preprocess'] = data['Review'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['preprocess']=data['preprocess'].apply(lambda x: BeautifulSoup(x).get_text())
data['preprocess']=data['preprocess'].apply(lambda x: re.sub(r"http\S+", "", x))
data['preprocess']=data['preprocess'].apply(lambda x: "".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))
data['preprocess']=data['preprocess'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))

C:\Users\coolz\anaconda3\lib\site-packages\bs4\__init__.py:431: MarkupResemblesLocatorWarning: "http://www.youtube.com/watch?v=o6gonrvww2y&feature=youtu.be" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
C:\Users\coolz\anaconda3\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: "........................... ............................ ............................ ................................" looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\coolz\anaconda3\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: ".." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\cool

In [26]:
X_train,X_test,Y_train, Y_test = train_test_split(data['preprocess'], data['label'], test_size=0.25, random_state=30)
print("Train:",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

Train: (135000,) (135000,) Test:  ((45000,), (45000,))


In [27]:
vectorizer = TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

In [37]:
classifier = LinearSVC(random_state=58, max_iter=100)

In [38]:
classifier.fit(tf_x_train,Y_train)

LinearSVC(max_iter=100, random_state=58)

In [39]:
y_test_pred = classifier.predict(tf_x_test)

In [40]:
report = classification_report(Y_test, y_test_pred)
print (report)

              precision    recall  f1-score   support

           0       0.87      0.13      0.23      7470
           1       0.85      1.00      0.92     37530

    accuracy                           0.85     45000
   macro avg       0.86      0.56      0.57     45000
weighted avg       0.85      0.85      0.80     45000



In [41]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test, y_test_pred)

0.5642133682612143